In [ ]:
import json
import shutil
import os
import numpy as np

# Extract Categories

In [ ]:
cat_ids = [2,3,4,6,8, 16,17,18, 27,28,31,33, 62,63,69, 73,77, 84] # [bird,cat,dog, backpack,umbrella,handbag,suitcase, chair,couch,dining table, ,laptop,cellphone,book]
new_ann_path = 'new_instances_train2017.json'
coco_json_path = 'instances_train2017.json'

f = open(coco_json_path)
data = json.load(f)

new_annotations = {}
# find images with certain categories
for ann in data['annotations']:
    if ann['category_id'] in cat_ids:
        image_id = str(ann['image_id'])        
        category_id = ann['category_id']
        bbox = ann['bbox']
        
        try:
            new_annotations[image_id][0]['category_id'].append(category_id)
            new_annotations[image_id][0]['bbox'].append(bbox)
        except KeyError:
            new_annotations[image_id] = []
            new_annotations[image_id].append({'category_id': [category_id], 'bbox': [bbox]})
            
# find the corosponding file names
for key in new_annotations.keys():
    for img in data['images']:
        if key == str(img['id']):
            new_annotations[key][0]['file_name'] = img['file_name']
            new_annotations[key][0]['height'] = img['height']
            new_annotations[key][0]['width'] = img['width']

json = json.dumps(new_annotations)
f = open(new_ann_path,"w")
f.write(json)

# Copy Images

In [ ]:
new_img_path = 'new_train'

f = open(new_ann_path)
data = json.load(f)
for e in data.values():
    print(e['file_name'])
    file_name = e['file_name']
    shutil.copy(os.path.join('train2017', file_name), os.path.join(new_img_path, file_name))

# Convert to YOLO

In [ ]:
for e in data.values():
    e = e[0]
    for i, bbox in enumerate(e['bbox']):
        xc = (bbox[0]+bbox[2])/e['width']
        yc = (bbox[1]+bbox[3])/e['height']
        width = bbox[2]/e['width']
        height = bbox[3]/e['height']
        
        e['bbox'][i] = [e['category_id'][i], xc, yc, width, height]
        
    np.savetxt(e['file_name'][:-4]+'.txt', e['bbox'], fmt='%.10g')